In [3]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [4]:
def comp_outside_good(data,name):
    #pre-processing to calculate outside good shares
    shares = data[['Market_ID',name]].copy()

    group_shares = shares.groupby('Market_ID').sum()
    group_shares['Outside Good Share'] = 1 - group_shares[name]

    data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')
    return data


data = pd.read_csv('data.csv')
data = comp_outside_good(data,'Inside Good Share')

In [5]:
#first estimate using logit
class logit(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(logit, self).__init__(*args, **kwds)

        
    def momcond(self, params):
        #unwrap stuff
        shares = np.array(self.endog).transpose()
        exog = np.array(self.exog)
        instr = np.array(self.instrument)
        
        lshare = np.log(shares[0]) -  np.log(shares[1])
        lshare = lshare.transpose()
       
        lshare_fit = np.matmul(exog,params) #linear equation    
        
        xi = lshare_fit - lshare
        g = instr * xi[:, np.newaxis]
        
        return g

# Question 1 - Estimates

I use dummy variables for the plan as my instrument. When regressing these dummy variables on plan, as a result my model has the form 

$$p_{jm} = p_j + \epsilon_{jm}$$ 

as is standard where $\epsilon_{jm}$ is a market specific shock. I estimated a model with the other exogeneous characteristics (i.e. Network score, Satisfaction, PPO) in the instrument. However, the model coefficients where similar, so for simplicitiy I use the dummy variables.

In [6]:
#calculate hausmann insturments
mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
hausman_instr = plan_dum

#set up x and y
y = data[['Inside Good Share','Outside Good Share']]
x =  data[['Network Score','Satisfaction Score','PPO','Premium']]

In [7]:
#set up initial est
beta_init = np.full(len(x.columns),1)

#set up model
model = logit(y , x, hausman_instr)

result = model.fit(beta_init, maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 292
         Function evaluations: 497
Optimization terminated successfully.
         Current function value: 0.005149
         Iterations: 139
         Function evaluations: 249
                                             logit Results                                             
Dep. Variable:     ['Inside Good Share', 'Outside Good Share']   Hansen J:                        16.99
Model:                                                   logit   Prob (Hansen J):                 0.108
Method:                                                    GMM                                         
Date:                                         Sat, 13 Oct 2018                                         
Time:                                                 16:34:52                                         
No. Observations:                                         3300                                  

# Question 2 - Elasticities and Markups

## Elasticities

Below are the formulas for elasticity

Own price : $$-\alpha (1-s_{jm}) p_{jm}$$
Cross price (good $j$, price $k$): $$-\alpha s_{km} p_{km}$$

The logit has a very inflexible substitution pattern. 

In [8]:
def comp_shares(x,p,alpha,beta,labels=False):
    #compute exp(delta_j)
    x = x.copy()
    x['exp_delta'] =  np.exp ( np.matmul(x[['Network Score','Satisfaction Score','PPO']],beta) - alpha*p )
    #print x['exp_delta']
    
    #compute 1 + sum_j exp(delta_j)
    sum_delta = x.groupby('Market_ID').sum()
    sum_delta['sum_exp_delta'] = 1 + sum_delta['exp_delta'] 
    
    x = pd.merge(x, sum_delta[['sum_exp_delta']], 
                right_index=True, left_on = 'Market_ID')

    #compute s_j
    x['fitted_share'] = x['exp_delta']/x['sum_exp_delta']
    
    if labels: 
        return x[['Plan_ID','Market_ID','fitted_share']]
    
    return np.array(x['fitted_share']).squeeze()
        

#set up parameters 
observ = data[['Plan_ID','Market_ID','Network Score','Satisfaction Score','PPO']]
prices = np.array( data['Premium'] ) #needed for later...

beta = result.params[:-1]
alpha = abs(result.params[3])

shares = comp_shares(observ,prices,alpha,beta)

In [9]:
def comp_elasticity(shares, prices, alpha):
    #set up matrix skeleton
    own_price = np.identity(len(shares))
    cross_price = 1 - own_price

    #actually calculate elasticity
    cross_elasticity = shares * alpha * prices
    own_elasticity  = -(1-shares) * alpha * prices

    return cross_price*cross_elasticity + own_price *own_elasticity


#agregate elasticities
elasticity = comp_elasticity(shares, prices, alpha)

#average elasticity
avg_shares =  np.array( comp_shares(observ,prices,alpha,beta,
                                    labels=True).groupby('Plan_ID').mean()['fitted_share']).squeeze()
avg_price = np.array( data[['Plan_ID','Premium']].groupby('Plan_ID').mean() ).squeeze()
avg_elasticity = comp_elasticity(avg_shares, avg_price, alpha)

Below are the model's implied elasticities

In [10]:
print pd.DataFrame(avg_elasticity)

          0         1         2         3         4         5         6   \
0  -4.326559  0.555455  1.089161  0.826840  0.571207  0.714491  1.002820   
1   1.015659 -4.308597  1.089161  0.826840  0.571207  0.714491  1.002820   
2   1.015659  0.555455 -4.287062  0.826840  0.571207  0.714491  1.002820   
3   1.015659  0.555455  1.089161 -4.377314  0.571207  0.714491  1.002820   
4   1.015659  0.555455  1.089161  0.826840 -4.252885  0.714491  1.002820   
5   1.015659  0.555455  1.089161  0.826840  0.571207 -4.141574  1.002820   
6   1.015659  0.555455  1.089161  0.826840  0.571207  0.714491 -4.347902   
7   1.015659  0.555455  1.089161  0.826840  0.571207  0.714491  1.002820   
8   1.015659  0.555455  1.089161  0.826840  0.571207  0.714491  1.002820   
9   1.015659  0.555455  1.089161  0.826840  0.571207  0.714491  1.002820   
10  1.015659  0.555455  1.089161  0.826840  0.571207  0.714491  1.002820   
11  1.015659  0.555455  1.089161  0.826840  0.571207  0.714491  1.002820   
12  1.015659

Note indexing starts at 0.

## Markups

The following formula relates marginal costs with prices:

$$\hat{mc}_i = pi - (\dfrac{ \partial s_i} { \partial p_i})^{-1} s_i$$

We can use it to calculate the markup

In [11]:
#solve for marginal costs
def comp_markup(shares):
    shares_vector = np.array([shares])
    
    #set up matrix    
    own_price = np.identity(len(shares))

    #caclulate formula
    own_deriv  = - alpha * (1-shares)  * shares
    
    derivative = own_price *own_deriv
    #take inverse and calc markup
    inv_derivative = np.linalg.inv(derivative)

    markup = - np.matmul(inv_derivative, shares_vector.transpose()) 
    return markup.transpose()[0]


markup = comp_markup(shares)
mc = prices - markup

data['Markup'] = markup
print data[['Plan_ID','Markup']].groupby('Plan_ID').mean()

           Markup
Plan_ID          
1        0.600929
2        0.548763
3        0.610518
4        0.578164
5        0.551221
6        0.570263
7        0.599144
8        0.572002
9        0.579314
10       0.536959
11       0.555631
12       0.598772
13       0.570590
14       0.562080
15       0.609716
16       0.564715


Overall the markups are lower than with the nested logit. It is possible this is related to the more strict substititon patterns imposed without the nest.

# Question 3 - Marginal Costs vs Plan Characteristics

In [12]:
model_q3 = sm.OLS(data['Premium'] - data['Markup'], data[['Network Score','Satisfaction Score','PPO']])
result_q3 = model_q3.fit()
print result_q3.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 6.770e+05
Date:                Sat, 13 Oct 2018   Prob (F-statistic):               0.00
Time:                        16:35:02   Log-Likelihood:                 3812.1
No. Observations:                3300   AIC:                            -7618.
Df Residuals:                    3297   BIC:                            -7600.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Network Score          1.9018      0

# Question 4  - Counterfactuals

In [ ]:
new_mc = mc +.25
new_prices = np.genfromtxt('prices.csv', delimiter=',') 

def comp_foc(p, costs = new_mc, x = observ, a = alpha, b = beta):
    shares = comp_shares(x,p,a,b)
    new_markup = comp_markup(shares)
    print new_markup - (p - costs)
    return new_markup - (p - costs)

new_prices = fsolve(comp_foc, new_prices,epsfcn = 1e-3,xtol=1e-3)
np.savetxt("prices2.csv", new_prices, delimiter=",")

In [ ]:
#iterate through to solve profits
new_prices = np.empty(len(prices))

for i in range(len(prices)): 
    
    def profits(p, all_prices=prices.copy(), x=observ, mc = mc, beta=beta, rebate=.25,):
        all_prices[i] = p
        shares = comp_shares(x, all_prices, alpha, beta)
        return - shares[i]*(p - mc[i] + rebate)      
    
    res = minimize(profits, prices[i], method='BFGS', options={'disp': False})
    new_prices[i]=res.x

new_prices = np.array(new_prices).squeeze()
np.savetxt("prices.csv", new_prices, delimiter=",")

In [15]:
new_prices = np.genfromtxt('prices.csv', delimiter=',') #avoid caclulating everytime

## Part 1 - Unisurance Rate

In [20]:
#outside good shares
new_data = data[['Market_ID']].copy()
new_data['New Inside Good'] =  comp_shares(observ, new_prices, alpha, beta)
new_data = comp_outside_good(new_data,'New Inside Good')

#compare the mean outside good before and after the rebate. It decreases.
print 'Outside Good (Before Rebate): %s'%data['Outside Good Share'].mean()
print 'Outside Good (After Rebate): %s'%new_data['Outside Good Share'].mean()

Outside Good (Before Rebate): 0.134505918182
Outside Good (After Rebate): 0.0978925678369


## Part 2 - Change in Profits

In [19]:
def industry_profits(x, p, alpha, beta, mc):
    """computes agregate profits"""
    shares = comp_shares(x, p, alpha, beta)
    return np.matmul(shares, np.array([prices - mc]).transpose())

#industry wide profits
print 'Industry Profits (Before Rebate): %s'%industry_profits(observ, prices, alpha, beta , mc)[0]
print 'Industry Profits (After Rebate): %s'%industry_profits(observ, new_prices, alpha, beta , mc)[0]
print '\n'

#profits per enrollee, comparision
print 'Per Enrollee (Before Rebate): %s'%(prices - mc).mean()
print 'Per Enrollee (After Rebate): %s'%(new_prices + .25 - mc).mean()

Industry Profits (Before Rebate): 297.54396984209365
Industry Profits (After Rebate): 313.4992352234833


Per Enrollee (Before Rebate): 0.5752567331174229
Per Enrollee (After Rebate): 0.6208741043229968


## Part 3 - Change in Consumer Surplus

As in Train Ch. 3 

$$\Delta E(CS) = \dfrac{1}{\alpha} [ln(\sum_{j} e^{\delta'_jm} ) - ln(\sum_{j} e^{\delta_jm} )  ]$$

In [18]:
def comp_surplus(x,p1,p2,alpha,beta,labels=False):
    #compute exp(delta_j)
    x = x.copy()
    x['exp_delta1'] =  np.exp ( np.matmul(x[['Network Score','Satisfaction Score','PPO']],beta) - alpha*p1 )
    x['exp_delta2'] =  np.exp ( np.matmul(x[['Network Score','Satisfaction Score','PPO']],beta) - alpha*p2 )
    
    #1/alpha *  ( sum(np.exp(delta1_j)) - sum(np.exp(delta0_j)) )
    utility_ratio = x['exp_delta2'].sum()/x['exp_delta1'].sum()
    return 1/alpha * np.log( utility_ratio )


print comp_surplus(x,prices,new_prices,alpha,beta)

0.20320784022358307
